In [ ]:
# importazione pacchetti necessari
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, accuracy_score
import os
import struct

# funzione per caricare il dataset MNIST
def load_mnist(path="/"):
    train_labels_path = os.path.join(path, "train-labels.idx1-ubyte")
    train_images_path = os.path.join(path, "train-images.idx3-ubyte")

    test_labels_path = os.path.join(path, "t10k-labels.idx1-ubyte")
    test_images_path = os.path.join(path, "t10k-images.idx3-ubyte")

    labels_path = [train_labels_path, test_labels_path]
    images_path = [train_images_path, test_images_path]

    labels = []
    images = []

    for path in zip(labels_path, images_path):

        with open(path[0], "rb") as lbpath:
            magic, n = struct.unpack(">II", lbpath.read(8))
            lb = np.fromfile(lbpath, dtype=np.uint8)
            labels.append(lb)

        with open(path[1], "rb") as imgpath:
            magic, num, rows, cols = struct.unpack(">IIII", imgpath.read(16))
            images.append(np.fromfile(imgpath, dtype=np.uint8).reshape(len(lb), 784))

    return images[0], images[1], labels[0], labels[1]

Suddivido il dataset nei vari subset di TRAIN e di TEST

In [ ]:
X_train, X_test, Y_train, Y_test = load_mnist(path="MNIST")

print("Numero caratteristiche per ogni immagine: " + str(X_train.shape[1]) + "\n")
print("Numero immagini nel set di TRAIN: " + str(X_train.shape[0]) + "\n")
print("Numero immagini nel set di TEST: " + str(X_test.shape[0]) + "\n")
print("Nomero totale di caratteristiche usate per addestrare: " + str(X_train.shape[1] * X_train.shape[0]) + "\n")
print("Nomero totale di caratteristiche da indovinare: " + str(X_train.shape[1] * X_test.shape[0]) + "\n")

# Visualizzazione di 36 immagini casuali dal dataset di training
plt.figure(figsize=(10,10))
random_inds = np.random.choice(len(X_train), 36, replace=False)
for i, image_ind in enumerate(random_inds):
    plt.subplot(6,6,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    image = X_train[image_ind].reshape(28, 28)
    plt.imshow(image, cmap=plt.cm.binary)
    plt.xlabel(Y_train[image_ind])
plt.show()

Standardizzo i valori delle caratteristiche in modo da portarli in un range tra 0 e 1

In [ ]:
# immagine non standardizzata
print("Singola immagine: " + str(X_train[0]) +"\n")

# standardizzazione dei dati
mms = MinMaxScaler()
X_train = mms.fit_transform(X_train)
X_test = mms.transform(X_test)

print("Immagine standardizzata: " + str(X_train[0]))

Dichiaro la rete neurale e i suoi parametri e la addestriamo sul set di TRAIN, sarà il nostro modello

In [ ]:
from sklearn.neural_network import MLPClassifier

# dichiaro i parametri delle rete (due hidden layers di cui uno con 100 nodi e il secondo con 30 nodi)
mlp = MLPClassifier(
    hidden_layer_sizes=(100, 30), verbose=True
)

# addestro il modello passandogli i dati di TRAIN
mlp.fit(X_train, Y_train)

Passo al modello, appena creato e addestrato, i dati del set di TEST per fargli fare le predizioni e vedere come li classifica

In [ ]:
# eseguio le predizioni sui dati di TEST
Y_pred = mlp.predict(X_test)
Y_proba = mlp.predict_proba(X_test)

print("Classificazione effettuata\n")

# calcolo l'accuratezza delle predizioni generate
acc = accuracy_score(Y_test, Y_pred)

# calcolo la loss
lloss = log_loss(Y_test, Y_proba)

print("Metriche calcolate e memorizzate")

Rifacciamo la stessa cosa ma passandogli i dati del set di TRAIN (ci serviranno unicamente come test per validare il risultato)

In [ ]:
# eseguo le predizioni sui dati di TRAIN per verificare overfitting
Y_pred_train = mlp.predict(X_train)
Y_proba_train = mlp.predict_proba(X_train)

# accuracy e loss
acc_train = accuracy_score(Y_train, Y_pred_train)
lloss_train = log_loss(Y_train, Y_proba_train)

Visualizzo e confronto i risultati generati dalle metriche utilizzate

In [ ]:
# stampo i risultati in console
print("-- Risultati metriche della Rete Neurale --\n")
print(f"Risultati sul set di TEST: Accuracy = {acc} / Loss = {lloss}\n")
print(f"Risultati sul set di TRAIN:  Accuracy = {acc_train} / Loss = {lloss_train}\n")
print(f"Differenza performance tra TEST e TRAIN: Accuracy = {abs(acc - acc_train)} / Loss = {abs(lloss - lloss_train)}\n")

Stampo gli elementi classificati in modo errato

In [ ]:
wrong_predictions = [i for i in range(len(Y_test)) if Y_test[i] != Y_pred[i]]
num_wrong_predictions = len(wrong_predictions)
max_errors_to_show = min(25, num_wrong_predictions)

if num_wrong_predictions == 0:
    print("Nessuna predizione errata")
else:
    print("Numero di predizioni errate: " + str(num_wrong_predictions))
    cols = 5
    rows = int(np.ceil(max_errors_to_show / cols))

    plt.figure(figsize=(2 * cols, 2 * rows))

    for index in range(max_errors_to_show):
        wrong_index = wrong_predictions[index]
        image = X_test[wrong_index].reshape(28, 28)
        plt.subplot(rows, cols, index + 1)
        plt.imshow(image, cmap="gray")
        plt.title(
            f"Reale: {Y_test[wrong_index]}, Pred: {Y_pred[wrong_index]}"
        )
        plt.axis("off")

    plt.tight_layout()
    plt.show()
